In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.1


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])




Epoch 1/10
     29/Unknown - 1s 50ms/step - loss: 13395.2051
Epoch 00001: loss improved from inf to 13395.20508, saving model to ./chkpt1
29/29 [==============================] - 1s 51ms/step - loss: 13395.2051
Epoch 2/10
28/29 [===========================>..] - ETA: 0s - loss: 12971.0303
Epoch 00002: loss improved from 13395.20508 to 12786.14062, saving model to ./chkpt1
29/29 [==============================] - 1s 40ms/step - loss: 12786.1406
Epoch 3/10
28/29 [===========================>..] - ETA: 0s - loss: 12628.9678
Epoch 00003: loss improved from 12786.14062 to 12509.63086, saving model to ./chkpt1
29/29 [==============================] - 1s 40ms/step - loss: 12509.6309
Epoch 4/10
28/29 [===========================>..] - ETA: 0s - loss: 12431.6025
Epoch 00004: loss improved from 12509.63086 to 12343.50684, saving model to ./chkpt1
29/29 [==============================] - 1s 40ms/step - loss: 12343.5068
Epoch 5/10
29/29 [==============================] - ETA: 0s - loss: 12230.3682

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 93s 451ms/step
Time taken: 100.90809726715088


(329.3206771699775,
 0.2039852313069685,
 0.12816811821117527,
 0.3548292396516293)

## Load Checkpoint and evaluate

In [5]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

loaded_model.load_weights('./chkpt1')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 93s 449ms/step
Time taken: 100.94414258003235


(329.3206771699775,
 0.2039852313069685,
 0.12816811821117527,
 0.3548292396516293)

# Training/eval with partition

In [3]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 896.0Bytes, after: 13.114MB, consumed: 13.113MB; exec time: 210.2s


In [4]:
optim = tf.optimizers.Adam(learning_rate=0.1, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

#model.compile(optimizer='rmsprop', loss=nll)
partitioned_model.compile(optimizer=optim, loss=nll)
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')
start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10, callbacks=[checkpoint])
print((time.time()-start))

Epoch 1/10
    274/Unknown - 10s 36ms/step - loss: 1422.5144
Epoch 00001: loss improved from inf to 1415.20312, saving model to ./chkpt1
277/277 [==============================] - 10s 38ms/step - loss: 1415.2031
Epoch 2/10
275/277 [============================>.] - ETA: 0s - loss: 1022.1102
Epoch 00002: loss improved from 1415.20312 to 1019.76959, saving model to ./chkpt1
277/277 [==============================] - 9s 34ms/step - loss: 1019.7696
Epoch 3/10
274/277 [============================>.] - ETA: 0s - loss: 838.1067
Epoch 00003: loss improved from 1019.76959 to 835.89728, saving model to ./chkpt1
277/277 [==============================] - 10s 35ms/step - loss: 835.8973
Epoch 4/10
276/277 [============================>.] - ETA: 0s - loss: 734.9833
Epoch 00004: loss improved from 835.89728 to 734.32019, saving model to ./chkpt1
277/277 [==============================] - 10s 35ms/step - loss: 734.3202
Epoch 5/10
275/277 [============================>.] - ETA: 0s - loss: 670.0822
Epo

In [5]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 119s 2s/step
Time taken: 122.69469952583313


(1331.8668137254901, 0.07005025369356002, 0.0, 0.1947794117647059)

## Save and Load checkpoint

In [6]:
partitioned_model.save_weights('./chkpt_partition')

In [7]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

loaded_part_model.load_weights('./chkpt_partition')

In [8]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=loaded_part_model.data_indexer)

start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 120s 2s/step
Time taken: 122.4106113910675


(1331.8668137254901, 0.07005025369356002, 0.0, 0.1947794117647059)

## Filtered evaluation

In [11]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=200,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
104/104 [==============================] - 711s 7s/step
Time taken: 769.5899913311005


(1181.0907133770427,
 0.13077588574264687,
 0.07300127214013113,
 0.24143751834817498)